In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-basis-constructor

# Introduction aux portes fractionnaires

*Estimation d'utilisation : moins de 30 secondes sur un processeur Heron r2 (REMARQUE : Il ne s'agit que d'une estimation. Votre temps d'exécution peut varier.)*
## Contexte
### Portes fractionnaires sur les QPU IBM
Les portes fractionnaires sont des portes quantiques paramétrées qui permettent l'exécution directe de rotations d'angle arbitraire (dans des limites spécifiques),
éliminant ainsi la nécessité de les décomposer en plusieurs portes de base.
En tirant parti des interactions natives entre les qubits physiques, les utilisateurs peuvent implémenter certaines unitaires plus efficacement sur le matériel.

Les QPU IBM Quantum&reg; Heron prennent en charge les portes fractionnaires suivantes :
- $R_{ZZ}(\theta)$ pour $0 < \theta < \pi / 2$
- $R_X(\theta)$ pour toute valeur réelle $\theta$

Ces portes peuvent réduire considérablement à la fois la profondeur et la durée des circuits quantiques.
Elles sont particulièrement avantageuses dans les applications qui reposent fortement sur $R_{ZZ}$ et $R_X$,
telles que la simulation hamiltonienne, l'algorithme d'optimisation approximative quantique (QAOA) et les méthodes de noyaux quantiques.
Dans ce tutoriel, nous nous concentrons sur le noyau quantique comme exemple pratique.
### Limitations

Les portes fractionnaires sont actuellement une fonctionnalité expérimentale et comportent quelques contraintes :
- $R_{ZZ}$ est limité aux angles dans la plage $0 < \theta < \pi / 2$.
- L'utilisation des portes fractionnaires n'est pas prise en charge pour les [circuits dynamiques](/guides/classical-feedforward-and-control-flow), le [Pauli twirling](/guides/error-mitigation-and-suppression-techniques#pauli-twirling), l'[annulation probabiliste d'erreurs](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec) (PEC) et l'[extrapolation à bruit nul](/guides/error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne) (ZNE) (utilisant l'[amplification probabiliste d'erreurs](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea) (PEA)).

Les portes fractionnaires nécessitent un flux de travail différent par rapport à l'approche standard.
Ce tutoriel explique comment travailler avec les portes fractionnaires à travers une application pratique.

Consultez les ressources suivantes pour plus de détails sur les portes fractionnaires.
- [Portes fractionnaires](/guides/fractional-gates)
- [Quand *ne pas* utiliser les portes fractionnaires](/guides/fractional-gates#when-not-to-use)
## Vue d'ensemble
Le flux de travail pour l'utilisation des portes fractionnaires suit généralement le flux de travail [Qiskit patterns](/guides/intro-to-patterns).
La différence principale est que tous les angles RZZ doivent satisfaire la contrainte $0 < \theta \leq \pi/2$.
Il existe deux approches pour garantir que cette condition est remplie.
Ce tutoriel se concentre sur la seconde approche et la recommande.

### 1. Générer des valeurs de paramètres qui satisfont la contrainte d'angle RZZ
Si vous êtes certain que tous les angles RZZ se situent dans la plage valide, vous pouvez suivre le flux de travail standard Qiskit patterns.
Dans ce cas, vous soumettez simplement les valeurs de paramètres dans le cadre d'un PUB. Le flux de travail se déroule comme suit.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import unitary_overlap
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

Si vous tentez de soumettre un PUB qui inclut une porte RZZ avec un angle en dehors de la plage valide, vous rencontrerez un message d'erreur tel que :
```
'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (20.0) outside of this range has been requested; via parameter value(s) γ[0]=10.0, substituted in parameter expression 2.0*γ[0].'
```
Pour éviter cette erreur, vous devriez envisager la seconde approche décrite ci-dessous.
### 2. Assigner les valeurs de paramètres aux circuits avant la transpilation
Le package `qiskit-ibm-runtime` fournit une passe de transpilation spécialisée appelée [`FoldRzzAngle`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/transpiler-passes-fold-rzz-angle).
Cette passe transforme les circuits quantiques de sorte que tous les angles RZZ respectent la contrainte d'angle RZZ.
Si vous fournissez le backend à `generate_preset_pass_manager` ou `transpile`, Qiskit applique automatiquement `FoldRzzAngle` aux circuits quantiques.
Cela vous demande d'assigner les valeurs de paramètres aux circuits quantiques avant la transpilation.
Le flux de travail se déroule comme suit.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=133
)  # backend should be a heron device or later
backend_name = backend.name
backend_c = service.backend(backend_name)  # w/o fractional gates
backend_f = service.backend(
    backend_name, use_fractional_gates=True
)  # w/ fractional gates
print(f"Backend: {backend_name}")
print(f"No fractional gates: {backend_c.basis_gates}")
print(f"With fractional gates: {backend_f.basis_gates}")
if "rzz" not in backend_f.basis_gates:
    print(f"Backend {backend_name} does not support fractional gates")

Backend: ibm_fez
No fractional gates: ['cz', 'id', 'rz', 'sx', 'x']
With fractional gates: ['cz', 'id', 'rx', 'rz', 'rzz', 'sx', 'x']


Notez que ce flux de travail entraîne un coût de calcul plus élevé que la première approche, car il implique l'assignation des valeurs de paramètres aux circuits quantiques et le stockage local des circuits liés aux paramètres.
De plus, il existe un problème connu dans Qiskit où la transformation des portes RZZ peut échouer dans certains scénarios. Pour une solution de contournement, veuillez consulter la section [Dépannage](#troubleshooting).
Ce tutoriel démontre comment utiliser les portes fractionnaires via la seconde approche à travers un exemple inspiré de la méthode des noyaux quantiques.
Pour mieux comprendre où les noyaux quantiques sont susceptibles d'être utiles, nous recommandons la lecture de [Liu, Arunachalam & Temme (2021).](https://www.nature.com/articles/s41567-021-01287-z)

Vous pouvez également suivre le tutoriel [Entraînement de noyaux quantiques](/tutorials/quantum-kernel-training) et la leçon [Noyaux quantiques](/learning/courses/quantum-machine-learning/quantum-kernel-methods) dans le cours Apprentissage automatique quantique sur IBM Quantum Learning.
### Prérequis
Avant de commencer ce tutoriel, assurez-vous d'avoir installé les éléments suivants :
- Qiskit SDK v2.0 ou version ultérieure, avec la prise en charge de la [visualisation](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.37 ou version ultérieure (`pip install qiskit-ibm-runtime`)
- Qiskit Basis Constructor (`pip install qiskit_basis_constructor`)
### Configuration

In [3]:
optimization_level = 2
shots = 2000
reps = 3
rng = np.random.default_rng(seed=123)

In [4]:
def my_zz_feature_map(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    x = ParameterVector("x", num_qubits * reps)
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    for k in range(reps):
        K = k * num_qubits
        for i in range(num_qubits):
            qc.rz(x[i + K], i)
        pairs = [(i, i + 1) for i in range(num_qubits - 1)]
        for i, j in pairs[0::2] + pairs[1::2]:
            qc.rzz((np.pi - x[i + K]) * (np.pi - x[j + K]), i, j)
    return qc


def quantum_kernel(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    qc = my_zz_feature_map(num_qubits, reps=reps)
    inner_product = unitary_overlap(qc, qc, "x", "y", insert_barrier=True)
    inner_product.measure_all()
    return inner_product


def random_parameters(inner_product: QuantumCircuit) -> np.ndarray:
    return np.tile(rng.random(inner_product.num_parameters // 2), 2)


def fidelity(result) -> float:
    ba = result.data.meas
    return ba.get_int_counts().get(0, 0) / ba.num_shots

Quantum kernel circuits and their corresponding parameter values are generated for systems with 4 to 40 qubits, and their fidelities are subsequently evaluated.

In [5]:
qubits = list(range(4, 44, 4))
circuits = [quantum_kernel(i, reps=reps) for i in qubits]
params = [random_parameters(circ) for circ in circuits]

## Flux de travail avec les portes fractionnaires
### Étape 1 : Mapper les entrées classiques sur un problème quantique
#### Circuit de noyau quantique
Dans cette section, nous explorons le circuit de noyau quantique utilisant des portes RZZ pour présenter le flux de travail des portes fractionnaires.

Nous commençons par construire un circuit quantique pour calculer les entrées individuelles de la matrice de noyau.
Cela se fait en combinant des circuits de carte de caractéristiques ZZ avec un recouvrement unitaire.
La fonction de noyau prend des vecteurs dans l'espace mappé par les caractéristiques et retourne leur produit scalaire comme entrée de la matrice de noyau :
$$K(x, y) = \langle \Phi(x) | \Phi(y) \rangle,$$
où $|\Phi(x)\rangle$ représente l'état quantique mappé par les caractéristiques.

Nous construisons manuellement un circuit de carte de caractéristiques ZZ utilisant des portes RZZ.
Bien que Qiskit fournisse un `zz_feature_map` intégré, celui-ci ne prend pas actuellement en charge les portes RZZ dans Qiskit v2.0.2 ([voir le problème](https://github.com/Qiskit/qiskit/issues/14469)).

Ensuite, nous calculons la fonction de noyau pour des entrées identiques - par exemple, $K(x, x) = 1$.
Sur des ordinateurs quantiques bruités, cette valeur peut être inférieure à 1 en raison du bruit.
Un résultat plus proche de 1 indique un niveau de bruit plus faible lors de l'exécution.
Dans ce tutoriel, nous désignons cette valeur comme la *fidélité*, définie par
$$\text{fidelity} = K(x, x).$$

In [6]:
circuits[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif" alt="Output of the previous code cell" />

In the standard Qiskit patterns workflow, parameter values are typically passed to the Sampler or Estimator primitive as part of a PUB.
However, when using a backend that supports fractional gates, these parameter values must be explicitly assigned to the quantum circuit prior to transpilation.

In [7]:
b_qc = [
    circ.assign_parameters(param) for circ, param in zip(circuits, params)
]
b_qc[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif" alt="Output of the previous code cell" />

Les circuits de noyau quantique et leurs valeurs de paramètres correspondantes sont générés pour des systèmes de 4 à 40 qubits, et leurs fidélités sont ensuite évaluées.

In [8]:
backend_f = service.backend(name=backend_name, use_fractional_gates=True)
# pm_f includes `FoldRzzAngle` pass
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_f
)

In [9]:
t_qc_f = pm_f.run(b_qc)
print(t_qc_f[0].count_ops())
t_qc_f[0].draw("mpl", fold=-1)

OrderedDict([('rz', 35), ('rzz', 18), ('x', 13), ('rx', 9), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif)

Dans le flux de travail standard Qiskit patterns, les valeurs de paramètres sont généralement transmises au primitive Sampler ou Estimator dans le cadre d'un PUB.
Cependant, lors de l'utilisation d'un backend prenant en charge les portes fractionnaires, ces valeurs de paramètres doivent être explicitement assignées au circuit quantique avant la transpilation.

In [10]:
nnl_f = [qc.num_nonlocal_gates() for qc in t_qc_f]
depth_f = [qc.depth() for qc in t_qc_f]
duration_f = [
    qc.estimate_duration(backend_f.target, unit="u") for qc in t_qc_f
]

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif)

### Étape 2 : Optimiser le problème pour l'exécution sur le matériel quantique

Nous transpilons ensuite le circuit en utilisant le gestionnaire de passes suivant le pattern standard de Qiskit.
En fournissant un backend prenant en charge les portes fractionnaires à `generate_preset_pass_manager`, une passe spécialisée appelée `FoldRzzAngle` est automatiquement incluse.
Cette passe modifie le circuit pour respecter les contraintes d'angle RZZ.
En conséquence, les portes RZZ avec des valeurs négatives dans la figure précédente sont transformées en valeurs positives, et quelques portes X supplémentaires sont ajoutées.

In [11]:
sampler_f = SamplerV2(mode=backend_f)
sampler_f.options.dynamical_decoupling.enable = True
sampler_f.options.dynamical_decoupling.sequence_type = "XY4"
sampler_f.options.dynamical_decoupling.skip_reset_qubits = True

In [12]:
job = sampler_f.run(t_qc_f, shots=shots)
print(job.job_id())

d4bninsi51bc738j97eg


### Step 4: Post-process and return result in desired classical format

You can obtain the kernel function value $K(x, x)$ by measuring the probability of the all-zero bitstring `00...00` in the output.

In [13]:
# job = service.job("d1obougt0npc73flhiag")
result = job.result()
fidelity_f = [fidelity(result=res) for res in result]
print(fidelity_f)
usage_f = job.usage()

[0.9005, 0.647, 0.3345, 0.355, 0.3315, 0.174, 0.1875, 0.149, 0.1175, 0.085]


![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif)

Pour évaluer l'impact des portes fractionnaires, nous mesurons le nombre de portes non locales (CZ et RZZ pour ce backend),
ainsi que les profondeurs et durées des circuits, et comparons ces métriques à celles d'un flux de travail standard plus loin.

In [14]:
# step 1: map classical inputs to quantum problem
# `circuits` and `params` from the previous section are reused here

In [15]:
# step 2: optimize circuits
backend_c = service.backend(backend_name)  # w/o fractional gates
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc_c = pm_c.run(circuits)
print(t_qc_c[0].count_ops())
t_qc_c[0].draw("mpl", fold=-1)

OrderedDict([('rz', 130), ('sx', 80), ('cz', 36), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a10f2d95-1.avif" alt="Output of the previous code cell" />

In [16]:
nnl_c = [qc.num_nonlocal_gates() for qc in t_qc_c]
depth_c = [qc.depth() for qc in t_qc_c]
duration_c = [
    qc.estimate_duration(backend_c.target, unit="u") for qc in t_qc_c
]

In [17]:
# step 3: execute
sampler_c = SamplerV2(backend_c)
sampler_c.options.dynamical_decoupling.enable = True
sampler_c.options.dynamical_decoupling.sequence_type = "XY4"
sampler_c.options.dynamical_decoupling.skip_reset_qubits = True

In [18]:
job = sampler_c.run(pubs=zip(t_qc_c, params), shots=shots)
print(job.job_id())

d4bnirvnmdfs73ae3a2g


In [19]:
# step 4: post-processing
# job = service.job("d1obp8j3rr0s73bg4810")
result = job.result()
fidelity_c = [fidelity(res) for res in result]
print(fidelity_c)
usage_c = job.usage()

[0.6675, 0.5725, 0.098, 0.102, 0.065, 0.0235, 0.006, 0.0015, 0.0015, 0.002]


## Comparaison du flux de travail et du circuit sans portes fractionnaires
Dans cette section, nous présentons le flux de travail standard Qiskit patterns utilisant un backend qui ne prend pas en charge les portes fractionnaires.
En comparant les circuits transpilés, vous remarquerez que la version utilisant les portes fractionnaires (de la section précédente) est plus compacte que celle sans portes fractionnaires.

In [20]:
plt.plot(qubits, depth_c, "-o", label="no fractional gates")
plt.plot(qubits, depth_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("depth")
plt.title("Comparison of depths")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/ef343a53-1.avif" alt="Output of the previous code cell" />

In [21]:
plt.plot(qubits, duration_c, "-o", label="no fractional gates")
plt.plot(qubits, duration_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("duration (µs)")
plt.title("Comparison of durations")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/98bb2cd0-1.avif" alt="Output of the previous code cell" />

In [22]:
plt.plot(qubits, nnl_c, "-o", label="no fractional gates")
plt.plot(qubits, nnl_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("number of non-local gates")
plt.title("Comparison of numbers of non-local gates")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/1383b242-1.avif" alt="Output of the previous code cell" />

In [23]:
plt.plot(qubits, fidelity_c, "-o", label="no fractional gates")
plt.plot(qubits, fidelity_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("fidelity")
plt.title("Comparison of fidelities")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/8b4594f5-1.avif" alt="Output of the previous code cell" />

We compare the QPU usage time with and without fractional gates. The results in the following cell show that the QPU usage times are almost identical.

In [24]:
print(f"no fractional gates: {usage_c} seconds")
print(f"fractional gates: {usage_f} seconds")

no fractional gates: 7 seconds
fractional gates: 7 seconds


## Advanced topic: Using only fractional RX gates

The need for the modified workflow when using fractional gates primarily stems from the restriction on RZZ gate angles.
However, if you use only the fractional RX gates and exclude the fractional RZZ gates, you can continue to follow the standard Qiskit patterns workflow.
This approach can still offer meaningful benefits, particularly in circuits that involve a large number of RX gates and U gates, by reducing the overall gate count and potentially improving performance.
In this section, we demonstrate how to optimize your circuits using only fractional RX gates, while omitting RZZ gates.

To support this, we provide a utility function that allows you to disable a specific basis gate in a Target object.
Here, we use it to disable RZZ gates.

In [25]:
from qiskit.circuit.library import n_local
from qiskit.transpiler import Target

In [26]:
def remove_instruction_from_target(target: Target, gate_name: str) -> Target:
    new_target = Target(
        description=target.description,
        num_qubits=target.num_qubits,
        dt=target.dt,
        granularity=target.granularity,
        min_length=target.min_length,
        pulse_alignment=target.pulse_alignment,
        acquire_alignment=target.acquire_alignment,
        qubit_properties=target.qubit_properties,
        concurrent_measurements=target.concurrent_measurements,
    )

    for name, qarg_map in target.items():
        if name == gate_name:
            continue
        instruction = target.operation_from_name(name)
        if qarg_map == {None: None}:
            qarg_map = None
        new_target.add_instruction(instruction, qarg_map, name=name)
    return new_target

We use a circuit consisting of U, CZ, and RZZ gates as an example.

In [27]:
qc = n_local(3, "u", "cz", "linear", reps=1)
qc.rzz(1.1, 0, 1)
qc.draw("mpl")

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6b812497-0.avif" alt="Output of the previous code cell" />

We first transpile the circuit for a backend that does not support fractional gates.

In [28]:
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc = pm_c.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 23), ('sx', 16), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/9e8e0709-1.avif" alt="Output of the previous code cell" />

## Comparaison des profondeurs et des fidélités
Dans cette section, nous comparons le nombre de portes non locales et les fidélités entre les circuits avec et sans portes fractionnaires.
Cela met en évidence les avantages potentiels de l'utilisation des portes fractionnaires en termes d'efficacité d'exécution et de qualité.

In [29]:
backend_f = service.backend(backend_name, use_fractional_gates=True)
target = remove_instruction_from_target(backend_f.target, "rzz")
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 22), ('sx', 14), ('cz', 4), ('rx', 1)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/db45feb0-1.avif" alt="Output of the previous code cell" />

### Optimize U gates with fractional RX gates

In this section, we demonstrate how to optimize U gates using fractional RX gates, building on the same circuit introduced in the previous section.

You will need to install the `qiskit-basis-constructor` [package](https://github.com/Qiskit/qiskit-basis-constructor) for this section.
This is a beta version of a new transpilation plugin for Qiskit, which might be integrated into Qiskit in the future.

In [30]:
# %pip install qiskit-basis-constructor

In [31]:
from qiskit.circuit.library import UGate
from qiskit_basis_constructor import DEFAULT_EQUIVALENCE_LIBRARY

We transpile the circuit using only fractional RX gates, excluding RZZ gates.
By introducing a custom decomposition rule, as shown in the following,
we can reduce the number of single-qubit gates required to implement a U gate.

This feature is currently under discussion in this [GitHub issue.](https://github.com/Qiskit/qiskit/issues/13455)

In [32]:
# special decomposition rule for UGate
x = ParameterVector("x", 3)
zxz = QuantumCircuit(1)
zxz.rz(x[2] - np.pi / 2, 0)
zxz.rx(x[0], 0)
zxz.rz(x[1] + np.pi / 2, 0)
DEFAULT_EQUIVALENCE_LIBRARY.add_equivalence(UGate(x[0], x[1], x[2]), zxz)

Next, we apply the transpiler using `constructor-beta` translation provided by `qiskit-basis-constructor` package.
As a result, the total number of gates is reduced compare to the previous transpilation.

In [33]:
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
    translation_method="constructor-beta",
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 16), ('rx', 9), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b19aae7c-1.avif" alt="Output of the previous code cell" />

## Troubleshooting

### Issue: Invalid RZZ angles might remain after transpilation

As of Qiskit v2.0.3, there are known issues where RZZ gates with invalid angles may remain in the circuits even after transpilation.
The issue typically arises under the following conditions.

#### Failure when using `target` option with `generate_preset_pass_manager` or `transpiler`

When the `target` option is used with `generate_preset_pass_manager` or `transpiler`, the specialized transpiler pass `FoldRzzAngle` is not invoked.
To ensure proper handling of RZZ angles for fractional gates, we recommend always using the `backend` option instead.
See [this issue](https://github.com/Qiskit/qiskit/issues/14318) for more details.

#### Failure when circuits contain certain gates

If your circuit includes certain gates such as `XXPlusYYGate`, the Qiskit transpiler may generate RZZ gates with invalid angles.
If you encounter this issue, see this [GitHub issue](https://github.com/Qiskit/qiskit-ibm-runtime/issues/2256#issuecomment-2889487152) for a workaround.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_cCNiGkGX5xZMzoG)